In [ ]:
!pip install -q google-cloud-translate
from google.cloud import translate
from google.colab import auth
PROJECT_ID = "translatehindi"
auth.authenticate_user(project_id=PROJECT_ID)

In [ ]:
!pip install -q transformers pandas numpy sentencepiece

In [ ]:
import numpy as np
import requests
import pandas as pd
import re
import time
import os
import requests
import csv
import string
from string import digits

In [ ]:
def clean(text):
    text=re.sub(r'[0-9] times',"",text)
    text=re.sub(r'\n{2,}',"\n\n",text)
    text=re.sub(r'--([A-Za-z0-9 ]+)--',"",text)
    text=re.sub("'", '', text)
    text=re.sub("www\..*",'',text)
    exclude = set(string.punctuation)
    text=''.join(ch for ch in text if ch not in set(string.punctuation))
    text=re.sub(r'\[|\(',"",text)
    text=re.sub(r'\]|\)',".",text)

    # text=re.sub(r'\n{2,}',".",text)
    # text=re.sub(r'(,\n)|,+|\n',",",text)
    # remove_digits = str.maketrans('', '', string.digits)
    # text=text.translate(remove_digits)

    text=re.sub(r"[२३०८१५७९४६x]", "", text)
    text=text.strip()
    text=re.sub(" +", " ", text)
    text=text.lower()
    # text='START_ '+ text + ' _END'
    return text
clean=np.vectorize(clean)

In [ ]:
df=pd.read_csv("/content/Lyrics_db1.csv",encoding = "utf-8")
df['English']=''
df['Hindi']=clean(df['Hindi'])

In [ ]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

M2M = M2M100ForConditionalGeneration.from_pretrained('facebook/m2m100_418M')
M2Mtokenizer = M2M100Tokenizer.from_pretrained('facebook/m2m100_418M',src_lang = "hi")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/272 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

MB = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
MBtokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt",src_lang='hi_IN')

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [ ]:
def llmtranslate(model,text):
  if model==M2M:
    tokenizer=M2Mtokenizer
    encoded_text = tokenizer(text, return_tensors="pt")
    generated_tokens = model.generate(**encoded_text, forced_bos_token_id=tokenizer.get_lang_id("en"))
  elif model==MB:
    tokenizer=MBtokenizer
    encoded_text = tokenizer(text, return_tensors="pt")
    generated_tokens = model.generate(**encoded_text,forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])
  return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

In [ ]:
def gpttranslate(text):
    headers = {
        # 'Content-Type': 'application/json',
        'Authorization': f"Bearer {OPENAI_API_KEY}",}
    json_data = {
        'model': 'gpt-3.5-turbo',
        'messages': [{'role': 'user','content': text+"\n\n Translate this song into English lamguage."}]}
    response = requests.post('https://api.openai.com/v1/chat/completions', headers=headers, json=json_data)
    newres=dict(response.json())
    time.sleep(15)
    return newres['choices'][0]['message']['content']

In [ ]:
def googletranslate(text):
    try:
      project_id="translatehindi"
      client = translate.TranslationServiceClient()
      location = "global"
      parent = f"projects/{project_id}/locations/{location}"

      response = client.translate_text(
          request={
              "parent": parent,
              "contents": [text],
              "mime_type": "text/plain",  # mime types: text/plain, text/html
              "source_language_code": "hi",
              "target_language_code": "en-US"})
      return "".join([i.translated_text for i in response.translations])
    except:
      return ""

from google.cloud import translate_v2

def googletranslate_v2(text: str) -> dict:
    translate_client = translate_v2.Client()
    if isinstance(text, bytes):
        text = text.decode("utf-8")

    result = translate_client.translate(text, target_language='en-US')
    # print("Text: {}".format(result["input"]))
    # print("Translation: {}".format(result["translatedText"]))
    # print("Detected source language: {}".format(result["detectedSourceLanguage"]))
    return result["translatedText"]

In [ ]:
# googletranslate=np.vectorize(googletranslate)
# df['English']=googletranslate(df['Hindi'])
# df.to_csv("output.csv")

In [ ]:
lyrics1=pd.read_csv("/content/lyrics1.csv",encoding = "utf-8",usecols=[1],header=0,names=["Hindi"])
lyrics1['Hindi']=clean(lyrics1['Hindi'])
lyrics2=pd.read_csv("/content/lyrics2.csv",encoding = "utf-8",usecols=[6],header=0,names=["Hindi"])
lyrics2['Hindi']=clean(lyrics2['Hindi'])

In [ ]:
import os
!pip install google-auth
!pip install requests
import requests
from google.auth.transport.requests import Request
idinfo = id_token.verify_oauth2_token(token, Request(), app.config["GOOGLE_APPLICATION_CREDENTIALS"])
data = {
  "source_language_code": "hi",
  "target_language_code": "en-US",
  "contents": lyrics1['Hindi'][0],
  "mime_type": "text/plain",
  "transliteration_config": { "enable_transliteration": True}
}
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/clientcreds.json'
response=greq.post("https://translation.googleapis.com/v3/projects/translatehindi/locations/global:translateText",json=data)

NameError: name 'id_token' is not defined